In [82]:
# import library
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import math as m
import scipy.stats as stats
class StockParameter:
	def __init__(self, stock, K, expiry, n, isCall, isEuropean):
		self.stock = stock
		self.K = K
		self.n = n
		self.isCall = isCall
		self.isEuropean = isEuropean
        
		self.wb_models = load_workbook('stockoptiondata.xlsx')
		self.ws_models = self.wb_models['Stock Option']
          
            
		self.today = datetime.now()
		self.T = (datetime.strptime(expiry, "%m-%d-%Y") - self.today).days / 365

		self.one_day_ago = self.today.replace(day = self.today.day - 5)
		self.one_year_ago = self.today.replace(year = self.today.year - 1)

		self.df = web.DataReader(self.stock, 'yahoo', self.one_year_ago, self.today)
		self.df = self.df.sort_values(by = "Date")
		self.df = self.df.dropna()

		
	def get_underlying(self):
		# Underlying price
		self.S0 = self.df['Close'].iloc[-1]
		print("Underlying: ", self.S0)
		return self.S0

	def get_sigma(self):
		# Previous day's price
		self.df = self.df.assign(close_day_before = self.df.Close.shift(1))

		# Stock Return
		self.df['returns'] = ((self.df.Close - self.df.close_day_before) / self.df.close_day_before)

		# Sigma
		self.sigma = np.sqrt(252) * self.df['returns'].std()
		print("Sigma: ", self.sigma)
		return self.sigma

	def get_risk_free(self):
		# Risk free rate
		self.r = web.DataReader("^TNX", 'yahoo', self.one_day_ago, self.today)['Close'].iloc[-1]/100
		print("Risk free rate: ", self.r)
		return self.r
        
	def _OutputModelPrices_(self):
		# Output model price to Excel file
		self.get_underlying()
		self.get_sigma()
		self.get_risk_free()
        
		self.ws_models['B4'] = self.S0
		self.ws_models['B5'] = self.K
		self.ws_models['B6'] = self.r
		self.ws_models['B7'] = 0
		self.ws_models['B8'] = self.T
		self.ws_models['B9'] = self.sigma
		self.ws_models['B10'] = self.n
		self.ws_models['B11'] = self.isCall
		self.ws_models['B12'] = self.isEuropean
		self.wb_models.save('stockoptiondata.xlsx')
        
if __name__ == "__main__":

	ibm = StockParameter("IBM", 150, "6-11-2021", 10, 1, 1)
	ibm._OutputModelPrices_()
    

Underlying:  147.4199981689453
Sigma:  0.2749718832864908
Risk free rate:  0.01559999942779541


In [83]:
import numpy as np
import math as m
import scipy.stats as stats
from openpyxl import Workbook, load_workbook


class StockOption:
	def __init__(self):
		self._DeleteWorkbookSheets_()  # To remove output sheets from previous runs

		self.CalcGreeks = 0

		# using openpyxl functionality
		self.wb_models = load_workbook('stockoptiondata.xlsx')
		self.ws_models = self.wb_models['Stock Option']
		self.S0 = self.ws_models['B4'].value
		self.K = self.ws_models['B5'].value
		self.r = self.ws_models['B6'].value
		self.div = self.ws_models['B7'].value
		self.T = self.ws_models['B8'].value
		self.sigma = self.ws_models['B9'].value
		# Ensure n has at least 1 time step
		self.n = max(1, self.ws_models['B10'].value)
		self.IsCall = self.ws_models['B9'].value
		self.IsEuropean = self.ws_models['B10'].value

	def _DeleteWorkbookSheets_(self):
		# utility function to remove output sheets from previous runs
		wb = load_workbook('stockoptiondata.xlsx')
		ws = wb["Stock Option"]
		ws.freeze_panes = None
		ws_active = wb['Stock Option']
		# delete other sheets but Stock Option data sheet
		for sheet in wb.worksheets:
			if sheet.title != 'Stock Option':
				wb.remove(sheet)

		for j in range(6, 18):
			for i in range(5, 7):
				ws.cell(i, j, 0.0)
		for j in range(6, 18):
			for i in range(8, 9):
				ws.cell(i, j, 0.0)

		wb.save("stockoptiondata.xlsx")


class BinomialOption(StockOption):
# 	def _init_(self):
#         # using openpyxl functionality
# 		self.wb_models = load_workbook('stockoptiondata.xlsx')
# 		self.ws_models = self.wb_models['Stock Option']
# 		self.S0 = self.ws_models['B4'].value
# 		print(self.S0)
# 		self.K = self.ws_models['B5'].value
# 		self.r = self.ws_models['B6'].value
# 		self.div = self.ws_models['B7'].value
# 		self.T = self.ws_models['B8'].value
# 		self.sigma = self.ws_models['B9'].value
# 		# Ensure n has at least 1 time step
# 		self.n = max(1, self.ws_models['B10'].value)
# 		self.IsCall = self.ws_models['B9'].value
# 		self.IsEuropean = self.ws_models['B10'].value
        
	def _InitParams_(self):
		self.dt = 0.0  # delta of time step, in years
		self.u = 0.0  # Up move increment
		self.d = 0.0  # Down move increment
		self.pu = 0.0  # Probability of Up move
		self.pd = 0.0  # Probability of Down move

	def _CRROptionPrice_(self):
		# Cox, Ross, Rubinstein (1979)
		self._InitParams_()
		self.dt = float(self.T)/float(self.n)  # delta of time step, in years
		self.u = np.exp(self.sigma*(np.sqrt(self.dt)))
		self.d = 1 / self.u
		self.pu = (np.exp((self.r - self.div)*self.dt) -
				   self.d) / (self.u - self.d)
		self.pd = 1 - self.pu
		self.model = 'CRR'

		if self.IsCall and self.IsEuropean:
			self.CRREuroCalloptionprice = round(self._CalcOptionPrice_(), 4)
		elif not self.IsCall and self.IsEuropean:
			self.CRREuroPutoptionprice = round(self._CalcOptionPrice_(), 4)
		elif self.IsCall and not self.IsEuropean:
			self.CRRAmerCalloptionprice = round(self._CalcOptionPrice_(), 4)
		else:
			self.CRRAmerPutoptionprice = round(self._CalcOptionPrice_(), 4)
		if self.CalcGreeks == 0:
			self._OutputModelTrees_()

	def _CalcOptionPrice_(self):
		self._StockPriceTree_()
		self._ProbabilityTree_()
		self._PayoffTree_()
		self._BackwardsDiscount_()
		return self.optionprice

	def _StockPriceTree_(self):
		self.StockTree = np.zeros((self.n+1, self.n+1))
		for j in range(self.n+1):
			for i in range(j+1):
				self.StockTree[i][j] = self.S0*(self.u**i)*(self.d**(j-i))

	def _ProbabilityTree_(self):
		self.ProbTree = np.zeros((self.n+1, self.n+1))
		for j in range(self.n+1):
			for i in range(j+1):
				  self.ProbTree[i][j] = m.factorial(j) / (m.factorial(i) *
				  m.factorial(j-i))\
						   * self.pu**i * self.pd**(j-i)

	def _PayoffTree_(self):
		if self.IsEuropean:
			self._EuropeanPayoffTree_()
		else:
			self._AmericanPayoffTree_()

	def _EuropeanPayoffTree_(self):
		# Initialize all payoff nodes to zeros
		self.PayoffTree = np.zeros((self.n+1, self.n+1))
		iopt = 0
		# Get payoffs at terminal nodes at the option maturity
		if self.IsCall:
			iopt = 1
		else:
			iopt = -1
		# Get payoffs at terminal nodes
		for i in range(self.n+1):
			self.PayoffTree[i][self.n] = np.maximum(0, iopt *
								(self.StockTree[i][self.n]-self.K))

	def _AmericanPayoffTree_(self):
		# Initialize all payoff nodes to zeros
		self.PayoffTree = np.zeros((self.n+1, self.n+1))
		iopt = 0
		if self.IsCall:
			iopt = 1
		else:
			iopt = -1
		# Get payoffs at each node
		for j in range(self.n+1):
			for i in range(j+1):
				self.PayoffTree[i][j] = np.maximum(
					0, iopt * (self.StockTree[i][j]-self.K))

	def _BackwardsDiscount_(self):
		if self.IsEuropean:
			TerminalPayoff = 0.0
			for i in range(self.n+1):
				TerminalPayoff = TerminalPayoff + self.PayoffTree[i][self.n] *\
						self.ProbTree[i][self.n]
			self.optionprice = TerminalPayoff * np.exp(-self.r * self.T)
		else:
		# Step backward through tree
			if self.IsCall:
				iopt = 1
			else:
				iopt = -1
			for j in range(self.n-1, -1, -1):
				for i in range(j+1):
					self.PayoffTree[i][j] = (self.pu * self.PayoffTree[i+1][j+1]
							+ self.pd * self.PayoffTree[i][j+1]) *\
							np.exp(-1 * (self.r - self.div) * self.dt)
					# Use Early exercise price
					self.PayoffTree[i][j] = np.maximum(iopt *
				(self.StockTree[i][j] - self.K), self.PayoffTree[i][j])
			self.optionprice = self.PayoffTree[0][0]

	def _BlackScholesOptionPrice_(self):    
		# Computed values for Black Scholes model
		print(self.S0)
		d1 = (np.log(self.S0 / self.K) + (((self.r - self.div) + 0.5 * \
							self.sigma**2) * self.T)) \
					/ (self.sigma * np.sqrt(self.T)) # d1
		d2 = (np.log(self.S0 / self.K) + (((self.r - self.div) - 0.5 * \
							self.sigma**2) * self.T)) \
					/ (self.sigma * np.sqrt(self.T)) # d2
		Nd1 = stats.norm.cdf(d1,0.0,1.0) # N(d1)
		Nd2 = stats.norm.cdf(d2,0.0,1.0) # N(d1)
		Nminusd1 = stats.norm.cdf(-d1,0.0,1.0) # N(-d1)
		Nminusd2 = stats.norm.cdf(-d2,0.0,1.0) # N(-d2)
		
		if self.IsCall: 
			self.BSEuroCalloptionprice = round(self.S0 * np.exp(-self.div * self.T) * Nd1 - \
				self.K * np.exp(-self.r * self.T) * Nd2,4)
		else:
			self.BSEuroPutoptionprice = round(self.K * np.exp(-self.r * self.T) * Nminusd2 - \
				self.S0 * np.exp(-self.div * self.T) * Nminusd1,4)
			  
	def _TheGreeks_(self):
	# The Greeks from revaluation after bumps in S0, sigma, r, T
		self.CalcGreeks = 1
		self._CalcDelta_()
		self._CalcVega_()
		self._CalcTheta_()
		self._CalcRho_()
		self._CalcGamma_()
		self._OutputModelGreeks_()
		self.CalcGreeks = 0
		
	def _CalcDelta_(self):
		szero = self.S0
		self.S0 = self.S0 + 1
		# European Call Delta
		self.IsCall, self.IsEuropean = 1, 1
		price = self.BSEuroCalloptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroCallDelta = self.BSEuroCalloptionprice - price
		self.BSEuroCalloptionprice = price
		
		price = self.CRREuroCalloptionprice
		self._CRROptionPrice_()
		self.CRREuroCallDelta = self.CRREuroCalloptionprice - price
		self.CRREuroCalloptionprice = price
		
		# European Put Delta
		self.IsCall, self.IsEuropean = 0, 1
		price = self.BSEuroPutoptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroPutDelta = self.BSEuroPutoptionprice - price
		self.BSEuroPutoptionprice = price
		
		price = self.CRREuroPutoptionprice
		self._CRROptionPrice_()
		self.CRREuroPutDelta = self.CRREuroPutoptionprice - price
		self.CRREuroPutoptionprice = price
		
		
		# American Call Delta
		self.IsCall, self.IsEuropean = 1, 0
		price = self.CRRAmerCalloptionprice
		self._CRROptionPrice_()
		self.CRRAmerCallDelta = self.CRRAmerCalloptionprice - price
		self.CRRAmerCalloptionprice = price
		
		# American Put Delta
		self.IsCall, self.IsEuropean = 0, 0
		price = self.CRRAmerPutoptionprice
		self._CRROptionPrice_()
		self.CRRAmerPutDelta = self.CRRAmerPutoptionprice - price
		self.CRRAmerPutoptionprice = price
		
	
	def _CalcGamma_(self):
		szero = self.S0
		
		# European Call Gamma
		self.IsCall, self.IsEuropean = 1, 1
		price0 = self.BSEuroCalloptionprice
		delta1 = self.BSEuroCallDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._BlackScholesOptionPrice_()
		price10 = self.BSEuroCalloptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._BlackScholesOptionPrice_()
		price11 = self.BSEuroCalloptionprice
		delta2 = price11 - price10
		self.BSEuroCallGamma = delta2 - delta1
		self.BSEuroCalloptionprice = price0
		
		price0 = self.CRREuroCalloptionprice
		delta1 = self.CRREuroCallDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._CRROptionPrice_()
		price10 = self.CRREuroCalloptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._CRROptionPrice_()
		price11 = self.CRREuroCalloptionprice
		delta2 = price11 - price10
		self.CRREuroCallGamma = delta2 - delta1
		self.CRREuroCalloptionprice = price0
		
		# European Put Gamma
		self.IsCall, self.IsEuropean = 0, 1
		price0 = self.BSEuroPutoptionprice
		delta1 = self.BSEuroPutDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._BlackScholesOptionPrice_()
		price10 = self.BSEuroPutoptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._BlackScholesOptionPrice_()
		price11 = self.BSEuroPutoptionprice
		delta2 = price11 - price10
		self.BSEuroPutGamma = delta2 - delta1
		self.BSEuroPutoptionprice = price0
		
		price0 = self.CRREuroPutoptionprice
		delta1 = self.CRREuroPutDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._CRROptionPrice_()
		price10 = self.CRREuroPutoptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._CRROptionPrice_()
		price11 = self.CRREuroPutoptionprice
		delta2 = price11 - price10
		self.CRREuroPutGamma = delta2 - delta1
		self.CRREuroPutoptionprice = price0
		
		# American Call Gamma
		self.IsCall, self.IsEuropean = 1, 0
		price0 = self.CRRAmerCalloptionprice
		delta1 = self.CRRAmerCallDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._CRROptionPrice_()
		price10 = self.CRRAmerCalloptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._CRROptionPrice_()
		price11 = self.CRRAmerCalloptionprice
		delta2 = price11 - price10
		self.CRRAmerCallGamma = delta2 - delta1
		self.CRRAmerCalloptionprice = price0
		
		# American Put Gamma
		self.IsCall, self.IsEuropean = 0, 0
		price0 = self.CRRAmerPutoptionprice
		delta1 = self.CRRAmerPutDelta
		self.S0 = self.S0 + 10 # add 10 dollars
		self._CRROptionPrice_()
		price10 = self.CRRAmerPutoptionprice
		self.S0 = self.S0 + 1 # add one more dollar
		self._CRROptionPrice_()
		price11 = self.CRRAmerPutoptionprice
		delta2 = price11 - price10
		self.CRRAmerPutGamma = delta2 - delta1
		self.CRRAmerPutoptionprice = price0

		self.S0 = szero
		
		
	def _CalcVega_(self):
		sigmazero = self.sigma
		self.sigma = self.sigma + .01
		# European Call Vega
		self.IsCall, self.IsEuropean = 1, 1
		price = self.BSEuroCalloptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroCallVega = self.BSEuroCalloptionprice - price
		self.BSEuroCalloptionprice = price
		
		price = self.CRREuroCalloptionprice
		self._CRROptionPrice_()
		self.CRREuroCallVega = self.CRREuroCalloptionprice - price
		self.CRREuroCalloptionprice = price
		
		# European Put Vega
		self.IsCall, self.IsEuropean = 0, 1
		price = self.BSEuroPutoptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroPutVega = self.BSEuroPutoptionprice - price
		self.BSEuroPutoptionprice = price
		
		price = self.CRREuroPutoptionprice
		self._CRROptionPrice_()
		self.CRREuroPutVega = self.CRREuroPutoptionprice - price
		self.CRREuroPutoptionprice = price
		
		# American Call Vega
		self.IsCall, self.IsEuropean = 1, 0
		price = self.CRRAmerCalloptionprice
		self._CRROptionPrice_()
		self.CRRAmerCallVega = self.CRRAmerCalloptionprice - price
		self.CRRAmerCalloptionprice = price
		
		# American Put Vega
		self.IsCall, self.IsEuropean = 0, 0
		price = self.CRRAmerPutoptionprice
		self._CRROptionPrice_()
		self.CRRAmerPutVega = self.CRRAmerPutoptionprice - price
		self.CRRAmerPutoptionprice = price

		self.sigma = sigmazero
		
	def _CalcTheta_(self):
		tzero = self.T
		self.T = self.T - 1/365
		# European Call Theta
		self.IsCall, self.IsEuropean = 1, 1
		price = self.BSEuroCalloptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroCallTheta = self.BSEuroCalloptionprice - price
		self.BSEuroCalloptionprice = price
		
		price = self.CRREuroCalloptionprice
		self._CRROptionPrice_()
		self.CRREuroCallTheta = self.CRREuroCalloptionprice - price
		self.CRREuroCalloptionprice = price
		
		# European Put Theta
		self.IsCall, self.IsEuropean = 0, 1
		price = self.BSEuroPutoptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroPutTheta = self.BSEuroPutoptionprice - price
		self.BSEuroPutoptionprice = price
		
		price = self.CRREuroPutoptionprice
		self._CRROptionPrice_()
		self.CRREuroPutTheta = self.CRREuroPutoptionprice - price
		self.CRREuroPutoptionprice = price
		
		# American Call Theta
		self.IsCall, self.IsEuropean = 1, 0
		price = self.CRRAmerCalloptionprice
		self._CRROptionPrice_()
		self.CRRAmerCallTheta = self.CRRAmerCalloptionprice - price
		self.CRRAmerCalloptionprice = price
		
		# American Put Theta
		self.IsCall, self.IsEuropean = 0, 0
		price = self.CRRAmerPutoptionprice
		self._CRROptionPrice_()
		self.CRRAmerPutTheta = self.CRRAmerPutoptionprice - price
		self.CRRAmerPutoptionprice = price

		self.T = tzero
			
	def _CalcRho_(self):
		rzero = self.r
		self.r = self.r + 0.01
		# European Call Rho
		self.IsCall, self.IsEuropean = 1, 1
		price = self.BSEuroCalloptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroCallRho = self.BSEuroCalloptionprice - price
		self.BSEuroCalloptionprice = price
		
		price = self.CRREuroCalloptionprice
		self._CRROptionPrice_()
		self.CRREuroCallRho = self.CRREuroCalloptionprice - price
		self.CRREuroCalloptionprice = price
		
		# European Put Rho
		self.IsCall, self.IsEuropean = 0, 1
		price = self.BSEuroPutoptionprice
		self._BlackScholesOptionPrice_()
		self.BSEuroPutRho = self.BSEuroPutoptionprice - price
		self.BSEuroPutoptionprice = price
		
		price = self.CRREuroPutoptionprice
		self._CRROptionPrice_()
		self.CRREuroPutRho = self.CRREuroPutoptionprice - price
		self.CRREuroPutoptionprice = price
		
		# American Call Rho
		self.IsCall, self.IsEuropean = 1, 0
		price = self.CRRAmerCalloptionprice
		self._CRROptionPrice_()
		self.CRRAmerCallRho = self.CRRAmerCalloptionprice - price
		self.CRRAmerCalloptionprice = price
		
		# American Put Rho
		self.IsCall, self.IsEuropean = 0, 0
		price = self.CRRAmerPutoptionprice
		self._CRROptionPrice_()
		self.CRRAmerPutRho = self.CRRAmerPutoptionprice - price
		self.CRRAmerPutoptionprice = price

		self.r = rzero
		
	def _OutputModelPrices_(self):
		# Output model price to Excel file
		self.ws_models['F5'] = self.BSEuroCalloptionprice
		self.ws_models['F6'] = self.CRREuroCalloptionprice


		self.ws_models['L5'] = self.BSEuroPutoptionprice
		self.ws_models['L6'] = self.CRREuroPutoptionprice


		self.ws_models['F8'] = self.CRRAmerCalloptionprice
		
		self.ws_models['L8'] = self.CRRAmerPutoptionprice
	
	def _OutputModelTrees_(self):
		# Output model trees to Excel file
		model = self.model
		if self.IsCall and self.IsEuropean:
			self.ws_trees = self.wb_models.create_sheet(model + 'EuroCall')
		elif not self.IsCall and self.IsEuropean:
			self.ws_trees = self.wb_models.create_sheet(model + 'EuroPut')
		elif self.IsCall and not self.IsEuropean:
			self.ws_trees = self.wb_models.create_sheet(model + 'AmerCall')
		else:
			self.ws_trees = self.wb_models.create_sheet(model + 'AmerPut')


		self.ws_trees.cell(1,1,str(model) + ' Stock Tree :: ' + \
											str(self.n) + ' Time Steps')
		for j in range(self.n+1):
			for i in range(j+1):
				if j == 1:
					for k in range(self.n+1):
						self.ws_trees.cell(self.n+1-k+1,j,k)
				self.ws_trees.cell(self.n+1-i+1,j+2,round(self.StockTree[i][j],4))
				self.ws_trees.cell(self.n+3,j+2,j)
		
		self.ws_trees.cell(self.n+5,1, str(model) + ' Probability Tree :: ' \
							+ str(self.n) + ' Time Steps')
		for j in range(self.n+1):
			for i in range(j+1):
				if j == 1:
					for k in range(self.n+1):
						self.ws_trees.cell(2*(self.n)+5-k+1,j,k)
				self.ws_trees.cell(2*(self.n)+5-i+1,j+2,round(self.ProbTree[i][j],4))
				self.ws_trees.cell(2*(self.n)+7,j+2,j)


		self.ws_trees.cell(2*(self.n)+9,1,str(model) + ' Payoff Tree :: ' +\
						str(self.n) + ' Time Steps')
		for j in range(self.n+1):
			for i in range(j+1):
				if j == 1:
					for k in range(self.n+1):
						self.ws_trees.cell(3*(self.n)+9-k+1,j,k)
				self.ws_trees.cell(3*(self.n)+9-i+1,j+2,round(self.PayoffTree[i][j],4))
				self.ws_trees.cell(3*(self.n)+11,j+2,j)
				
				
	def _OutputModelGreeks_(self):
		# Output model price to Excel file
		# European Call Delta
		self.ws_models['G5'] = self.BSEuroCallDelta
		self.ws_models['G6'] = self.CRREuroCallDelta
		# European Put Delta
		self.ws_models['M5'] = self.BSEuroPutDelta
		self.ws_models['M6'] = self.CRREuroPutDelta
		# American Call Delta
		self.ws_models['G8'] = self.CRRAmerCallDelta
		# American Put Delta
		self.ws_models['M8'] = self.CRRAmerPutDelta
		
		# European Call Gamma
		self.ws_models['H5'] = self.BSEuroCallGamma
		self.ws_models['H6'] = self.CRREuroCallGamma
		# European Put Gamma
		self.ws_models['N5'] = self.BSEuroPutGamma
		self.ws_models['N6'] = self.CRREuroPutGamma
		# American Call Gamma
		self.ws_models['H8'] = self.CRRAmerCallGamma
		# American Put Gamma
		self.ws_models['N8'] = self.CRRAmerPutGamma
		
		# European Call Vega
		self.ws_models['I5'] = self.BSEuroCallVega
		self.ws_models['I6'] = self.CRREuroCallVega
		# European Put Vega
		self.ws_models['O5'] = self.BSEuroPutVega
		self.ws_models['O6'] = self.CRREuroPutVega
		# American Call Vega
		self.ws_models['I8'] = self.CRRAmerCallVega
		# American Put Vega
		self.ws_models['O8'] = self.CRRAmerPutVega
		
		# European Call Theta
		self.ws_models['J5'] = self.BSEuroCallTheta
		self.ws_models['J6'] = self.CRREuroCallTheta
		# European Put Theta
		self.ws_models['P5'] = self.BSEuroPutTheta
		self.ws_models['P6'] = self.CRREuroPutTheta
		# American Call Theta
		self.ws_models['J8'] = self.CRRAmerCallTheta
		# American Put Theta
		self.ws_models['P8'] = self.CRRAmerPutTheta
		
		# European Call Rho
		self.ws_models['K5'] = self.BSEuroCallRho
		self.ws_models['K6'] = self.CRREuroCallRho
		# European Put Rho
		self.ws_models['Q5'] = self.BSEuroPutRho
		self.ws_models['Q6'] = self.CRREuroPutRho
		# American Call Rho
		self.ws_models['K8'] = self.CRRAmerCallRho
		# American Put Rho
		self.ws_models['Q8'] = self.CRRAmerPutRho
		
	def PriceAll(self):
		self.IsCall = 1
		self.IsEuropean = 1
		
		while self.IsCall and self.IsEuropean:
			self._BlackScholesOptionPrice_()
			self._CRROptionPrice_()
			self.IsCall = 0
			
		while not self.IsCall and self.IsEuropean:
			self._BlackScholesOptionPrice_()
			self._CRROptionPrice_()
			self.IsCall = 1
			self.IsEuropean = 0
			
		while self.IsCall and not self.IsEuropean:
			self._CRROptionPrice_()
			self.IsCall = 0
		
		while not self.IsCall and not self.IsEuropean:
			self._CRROptionPrice_()
			break
		self._OutputModelPrices_()
		self.wb_models.save('stockoptiondata.xlsx')
		self._TheGreeks_()
		
		# Save your file to commit your calculations from memory to your Excel file
		# Your Excel file must be closed or you will get 'Permission Denied' to write
		self.wb_models.save('stockoptiondata.xlsx')
	
if __name__ == "__main__":

	binomialoption = BinomialOption()
	binomialoption.PriceAll()


147.4199981689453
147.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
148.4199981689453
158.4199981689453
159.4199981689453
180.4199981689453
181.4199981689453
